<a href="https://colab.research.google.com/github/XavierCarrera/text-classification-system/blob/main/MEMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

Cloning into 'UD_Spanish-AnCora'...
remote: Enumerating objects: 23, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 518 (delta 10), reused 18 (delta 7), pack-reused 495
Receiving objects: 100% (518/518), 115.95 MiB | 16.26 MiB/s, done.
Resolving deltas: 100% (357/357), done.


In [3]:
from conllu import parse_incr

tagtype = 'upos'
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", 'r', encoding='utf-8')

# transform data_file type conllu into an iterable list to use i and i+1 
word_list = []

for tokenlist in parse_incr(data_file):
    paragraph_list = []
    for token in tokenlist:
        paragraph_list.append((token['form'],token[tagtype]))
    word_list.append(paragraph_list)

#
postDict = {}
transitions = 0

for lst in word_list:
    for i in range(len(lst)):
        transitions += 1
        if i+1 == len(lst):
            tag = lst[i][1]
            word = lst[i][0].lower()
            post_tag = 'None'

            complet_tag = tag + '-' + word + '|' + post_tag         
            if complet_tag in postDict.keys():
                postDict[complet_tag] = postDict[complet_tag] + 1
            else:
                postDict[complet_tag] = 1
            break

        else:    
            tag = lst[i][1]
            word = lst[i][0].lower()
            post_tag = lst[i+1][1]
            complet_tag = tag + '-' + word + '|' + post_tag              
            if complet_tag in postDict.keys():
                postDict[complet_tag] = postDict[complet_tag] + 1
            else:
                postDict[complet_tag] = 1

print(transitions)
print(len(postDict))
print(postDict)

52511
16719
{'DET-el|NOUN': 1091, 'NOUN-gobernante|PUNCT': 1, 'PUNCT-,|ADP': 522, 'ADP-con|ADJ': 10, 'ADJ-ganada|NOUN': 1, 'NOUN-fama|ADP': 1, 'ADP-desde|SCONJ': 7, 'SCONJ-que|VERB': 100, 'VERB-llegó|VERB': 1, 'VERB-hace|NUM': 11, 'NUM-16|NOUN': 4, 'NOUN-meses|ADP': 8, 'ADP-al|NOUN': 172, 'NOUN-poder|ADP': 4, 'ADP-de|VERB': 121, 'VERB-explotar|ADP': 1, 'NOUN-máximo|DET': 1, 'DET-su|NOUN': 288, 'NOUN-oratoria|CCONJ': 1, 'CCONJ-y|ADJ': 97, 'ADJ-acusado|ADP': 3, 'ADP-por|DET': 247, 'DET-sus|NOUN': 115, 'NOUN-detractores|ADP': 2, 'ADP-de|NOUN': 994, 'NOUN-incontinencia|ADJ': 1, 'ADJ-verbal|PUNCT': 1, 'PUNCT-,|VERB': 349, 'VERB-enmudeció|ADP': 1, 'ADP-desde|DET': 31, 'NOUN-momento|ADP': 13, 'ADP-en|DET': 770, 'DET-el|PRON': 40, 'PRON-que|DET': 48, 'DET-el|PROPN': 298, 'PROPN-tribunal|PROPN': 3, 'PROPN-supremo|ADP': 1, 'ADP-de|PROPN': 585, 'PROPN-justicia|PUNCT': 4, 'PUNCT-(|PROPN': 119, 'PROPN-tsj|PUNCT': 1, 'PUNCT-)|VERB': 20, 'VERB-decidió|VERB': 3, 'VERB-suspender|ADV': 1, 'ADV-temporalm

In [4]:
postProbDict = {}
for key in postDict.keys():
    postProbDict[key] = postDict[key]/transitions

#Verification -> P(postProbDict) = 1
temp=0
for key in postProbDict.keys():
    temp= temp + postProbDict[key]
print(f'P(postProbDict) = {temp}')

P(postProbDict) = 0.9999999999994472


In [5]:
state_set = set()
for key in postDict.keys():
    w = key.split('|')[0].split('-')[0]
    state_set.add(w)

tag_state_dict = {}
for i, state in enumerate(state_set):
    tag_state_dict[state] = i
tag_state_dict

{'ADJ': 8,
 'ADP': 14,
 'ADV': 13,
 'AUX': 7,
 'CCONJ': 4,
 'DET': 0,
 'INTJ': 15,
 'NOUN': 10,
 'NUM': 3,
 'PART': 12,
 'PRON': 5,
 'PROPN': 6,
 'PUNCT': 16,
 'SCONJ': 1,
 'SYM': 9,
 'VERB': 2,
 '_': 11}

In [12]:
from conllu import parse_incr

wordlist = []
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", 'r', encoding='utf-8')
counter = 0 
initTagStateProb = {} # rho[i]

for tokenlist in parse_incr(data_file):
    counter+=1
    tag = tokenlist[0]['upos']
    if tag in initTagStateProb.keys():
        initTagStateProb[tag] += 1
    else:
        initTagStateProb[tag] = 1
for key in initTagStateProb.keys():
    initTagStateProb[key] /=counter

print(initTagStateProb)

#Verification -> P(initTagStateProb) = 1
temp=0
for key in initTagStateProb.keys():
    temp= temp + initTagStateProb[key]
print(f'P(initTagStateProb) = {temp}')

{'DET': 0.3633615477629988, 'PROPN': 0.1124546553808948, 'ADP': 0.16384522370012092, 'PRON': 0.034461910519951636, 'SCONJ': 0.02418379685610641, 'ADV': 0.06287787182587666, 'PUNCT': 0.07799274486094317, 'VERB': 0.04353083434099154, 'ADJ': 0.010882708585247884, 'CCONJ': 0.03325272067714631, 'NOUN': 0.02720677146311971, '_': 0.0006045949214026602, 'INTJ': 0.0006045949214026602, 'AUX': 0.022370012091898428, 'NUM': 0.01995163240628779, 'SYM': 0.0006045949214026602, 'PART': 0.0018137847642079807}
P(initTagStateProb) = 1.0


In [13]:
import numpy as np
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [14]:
def memm(sequence,
         postDict=postDict,
         postProbDict=postProbDict,
         tag_state_dict=tag_state_dict,
         initTagStateProb=initTagStateProb):
    
    #tokenization
    seq = word_tokenize(sequence)
    
    #Viterbi matrix inizialitations
    viterbi_matrix = np.zeros((17, len(seq)))

    #First Viterbi column Probability
    for key in tag_state_dict.keys():
        tag_row = tag_state_dict[key]
        word_tag = key+'-'+seq[0].lower()
        # print(word_tag)
        
        for key2 in tag_state_dict.keys():
            complete_tag = word_tag+'|'+key2
            if complete_tag in postProbDict.keys():
                viterbi_matrix[tag_row, 0] = initTagStateProb[key]*postProbDict[complete_tag]
    
    for col in range(1,len(seq)):
        for key in tag_state_dict.keys():
            tag_row = tag_state_dict[key]
            word_tag = key + '-' + seq[col]
            for key2 in tag_state_dict.keys():
                complete_tag = word_tag + '|' + key2
                if complete_tag in postProbDict.keys():
                    viterbi_matrix[tag_row, col] = max(viterbi_matrix[:,col-1])*postProbDict[complete_tag]

    res = []
    for i, p in enumerate(seq):
        for tag in tag_state_dict.keys():
            if tag_state_dict[tag] == np.argmax(viterbi_matrix[:, i]):
                res.append((p,tag))

    return res


    # print(viterbi_matrix)


tags = memm('el mundo es pequeño')
tags

[('el', 'DET'), ('mundo', 'NOUN'), ('es', 'AUX'), ('pequeño', 'ADJ')]

In [17]:
memm('el hombre en el castillo')

[('el', 'DET'),
 ('hombre', 'NOUN'),
 ('en', 'ADP'),
 ('el', 'DET'),
 ('castillo', 'PROPN')]

In [18]:
memm('el señor de los anillos')

[('el', 'DET'),
 ('señor', 'NOUN'),
 ('de', 'ADP'),
 ('los', 'DET'),
 ('anillos', 'PROPN')]